# Probability of Detection of Malcode Bytes in a Page

Suppose we have an $n$ element array, and there are $k$ _maliciously altered_ bytes.  How likely are we to detect one of these bytes if we sample $s$ times?

If we are sampling with replacement, then we use the Binomial distribution.  If we are sampling _without_ replacement, then we use the hypergeometric distribution.

Let's assume we are sampling _with_ replacement (IID), since it is just simpler to do.  Each sample is a Bernoulli trial (either we hit the malicious byte range or we do not).  Let's call detecting (sampling) a malicious byte a _failure_ and missing it a _success_.  The probability of a failure (detection) is $k/n$ and the probability of a success is $(n-k)/n$.

You see $f$ failures among $s$ trials with the following probability.
$$ P = {s\choose f} \left( \frac{k}{n} \right)^f \left( \frac{n-k}{n} \right)^{s-f} $$

The probability of observing _no_ failures and thus _not detecting_ the malcode is thus given by the following.
$$ \begin{array}{rl}
  P & = {s\choose 0} \left( \frac{k}{n} \right)^0 \left( \frac{n-k}{n} \right)^{s-0} \\
    & = \left( \frac{n-k}{n} \right)^s
\end{array} $$

(This isn't surprising and could be derived directly; it is just worth pointing out.)

We can now compute the probability of detection of one or more of the malcode bytes as $1-P$.  The following code computes this value.

In [1]:
import math

class Detector(object):
    def __init__(self, n, k):
        '''Create a new detector instance.  Page size is n, size of malcode is k.'''
        self.n = n
        self.k = k
        
    def __call__(self, s):
        '''Compute the probability of detecting the malcode given s samples.'''
        prob = math.pow((self.n-self.k)/self.n, s)
        return 1.0 - prob

Now we can use it to check a range of values.  Suppose we want to detect a 100-byte malcode sequence in a single 4KB page (4,096 bytes).  Because we are sampling with replacement, the value approaches 1.0 asymptotically.  (If we used the hypergeometric distribution for sampling without replacement, we would eventually reach 1.0 because we would sample every byte.)

In [2]:
detect = Detector(4096, 100)
for trials in range(10,110,10):
    print(f"For {trials} trials, probability of detection is {detect(trials)}")

For 10 trials, probability of detection is 0.2189922995883098
For 20 trials, probability of detection is 0.39002697189764357
For 30 trials, probability of detection is 0.5236063680086234
For 40 trials, probability of detection is 0.6279329049876419
For 50 trials, probability of detection is 0.7094127337255405
For 60 trials, probability of detection is 0.7730491073980648
For 70 trials, probability of detection is 0.8227496052625821
For 80 trials, probability of detection is 0.8615660768090649
For 90 trials, probability of detection is 0.8918820399896793
For 100 trials, probability of detection is 0.9155590406791363
